### Python notebook walking through considerations for choosing the RC lengths, and folding mirror curvatures, for the 40m Ponderomotive Squeezing Experiment

#### Resonance conditions:
 - $f_{1}$ = 11.066209 MHz (set by IMC length)
 - $f_{2}$ = 5  $\times  f_{1}$ 
 - Carrier, $f_{1}$, and $f_{2}$ sidebands have to be simultaneously resonant in the PRC.
 - Carrier and $f_{2}$ sidebands have to be simultaneously resonant (when the arms are resonant), while $f_{1}$ has to be non-resonant in the SRC (for ** detuned ** setup).
 - The coupling of the $f_{2}$ sideband to the SRC is set by the Schnupp asymmetry. We'd like to choose the Schnupp asymmetry to maximize this coupling (ideally we realize critical coupling from the PRC to the SRC).
 - In all of the above, the output coupler for the PRC and the input coupler for the SRC is the compound mirror formed by BS and the two ~37.79m long arm cavities. In fact, $\mathrm{L_{arm}}$ is tweaked to allow the simultaneous resonance of the $f_{1}$ and $f_{2}$ sidebands in the PRC.
 - The $f_{1}$ and $f_{2}$ sidebands have to be anti-resonant (or nearly so) in the arm cavities.

Having established the resonance requirements, all that needs to be done is to choose lengths for the RCs such that the round-trip phase gain by the $\mathrm{TEM_{00}}$ modes of the resonant fields is $2\pi$, while avoiding this condition for the non-resonant fields.

Note: Based on the resonance formulae above: 
 - Shortest candidate PRC lengths are: 6.77m, 20.32m, 33.86m.
 - Shortest candidate SRC lengths are: 1.35m, 4.06m and 9.48m (6.77m is not a great option because then the f1 sideband will also be resonant).


In [ ]:
# Imports
import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as scc
import matplotlib.colors as colors
from matplotlib.ticker import FormatStrFormatter
import matplotlib as mpl
from IPython.display import SVG
import emcee
import corner

#### Interferometer topology
The DRFPMI topology is shown below. 
 - The FPMI reflectivity is calculated with the arms as compound mirrors.
 - IFO reflection is E2/E1
 - Transmission to AS port is E11/E1
 - The adjacency matrix is written down using the graphic below, and solved for the IFO reflection and transmission to AS port [See Adjacency_DRFPMI_GV.nb]

In [ ]:
SVG(filename='./IFOschematic.svg')

In [ ]:
# Set up some plotting stuff
if 'gvELOG' in plt.style.available:
    plt.style.use('gvELOG')
else:
    mpl.rcParams['lines.linewidth'] = 3
    mpl.rcParams['lines.markersize'] = 10
    mpl.rcParams['axes.titlesize'] = 18
    mpl.rcParams['axes.labelsize'] = 16
    mpl.rcParams['xtick.labelsize'] = 16
    mpl.rcParams['ytick.labelsize'] = 16
    mpl.rcParams['axes.formatter.limits'] = [-2,2]
    mpl.rcParams['axes.grid'] = True
    mpl.rcParams['grid.linestyle'] = '--'
    mpl.rcParams['grid.linewidth'] = 0.7
    mpl.rcParams['grid.alpha'] = 0.4
    mpl.rcParams['text.usetex'] = True
    mpl.rcParams['font.family'] = 'sans-serif'
    mpl.rcParams['font.style'] = 'normal'
    mpl.rcParams['font.weight'] = 'extra bold'
    mpl.rcParams['font.size'] = 16
saveFigs=False

#### Choosing $\mathrm{L_{arm}}$
 - We require both $f_{1}$ and $f_{2}$ sidebands to be resonant inside the PRC, but nearly anti-resonant in the arm cavities.
 - One way to achieve this is if the phase change on reflection from the FP arm cavity is 5 $\times$ for the $f_{2}$ sideband as compared to the $f_{1}$ sideband.
 - If the * amplitude * reflectivity of the arm is $\mathrm{r_{arm}}$, then $\vec{\mathrm{E}}_{\mathrm{refl}} = \mathrm{r_{arm}}\vec{\mathrm{E}}_{\mathrm{inc}}$.
 - So the phase gained during the reflection process is the argument of the complex number
 $$\mathrm{r_{arm}} = \frac{-r_{\mathrm{ITM}} + r_{\mathrm{ETM}}(t_{\mathrm{ITM}}^2 + r_{\mathrm{ITM}}^2)e^{-i \frac{\omega}{\nu_{\mathrm{FSR}}}}}{1 - r_{\mathrm{ITM}}r_{\mathrm{ETM}}e^{-i \frac{\omega}{\nu_{\mathrm{FSR}}}}}.$$
 - Note that $\frac{\omega}{\nu_{\mathrm{FSR}}} = \frac{2 \omega L}{c} = 2\phi$ so we recover the more conventional expression of $\mathrm{r_{arm}} = \frac{-r_{\mathrm{ITM}} + r_{\mathrm{ETM}}(t_{\mathrm{ITM}}^2 + r_{\mathrm{ITM}}^2)e^{-2i \phi}}{1 - r_{\mathrm{ITM}}r_{\mathrm{ETM}}e^{-2i \phi}}$ where $\phi = \frac{\omega L}{c}$ is the one-way propagation phase.

In [ ]:
# 40m IFO parameters (c.f. 40m wiki, core optics page)
# Test masses
T_ITM = 1.384e-2
T_ETM = 13.7e-6
L_ITM = 20e-6 # losses, ppm
L_ETM = 20e-6
L_arm = 37.79 # meters
r_ITM = np.sqrt(1 - T_ITM - L_ITM)
t_ITM = np.sqrt(T_ITM)
r_ETM = np.sqrt(1 - T_ETM - L_ETM)
t_ETM = np.sqrt(T_ETM)

# Power Recycling
T_PRM = 1e-2
L_PRM = 50e-6
t_PRM = np.sqrt(T_PRM)
r_PRM = np.sqrt(1-T_PRM-L_PRM)

# Signal Recycling
T_SRM = 25e-2
L_SRM = 50e-6
t_SRM = np.sqrt(T_SRM)
r_SRM = np.sqrt(1-T_SRM-L_SRM)

# Modulation
f1 = 11.066209e6
f2 = 5*f1

def rArm(ti, te, ri, re, L, f):
    '''
    Computes reflectivity of a resonant FP cavity 
    for a sideband offset at frequency f.
    '''
    FSR = scc.c / (2*L) 
    return (-ri + re*(ti**2 + ri**2)*np.exp(-1j*2*np.pi*f/FSR))/(1 - ri*re*np.exp(-1j*2*np.pi*f/FSR))
lengths = np.linspace(35,45,20000)
rr_f1 = rArm(t_ITM,t_ETM, r_ITM, r_ETM, lengths, f1)
rr_f2 = rArm(t_ITM,t_ETM, r_ITM, r_ETM, lengths, f2)
ang_f1 = np.angle(-rr_f1, deg=True) # minus sign is for scaling the plot...
ang_f2 = np.angle(-rr_f2, deg=True)
argmin = np.argmin(np.abs(ang_f1 - ang_f2/5)),
print('Argmin is {} corresponding to L= {:.4f} m, yielding dPhi = {:.4f} degrees'.format(argmin,
     lengths[argmin], ang_f1[argmin] - ang_f2[argmin]/5))

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True,figsize=(12,8))
ax[0].plot(lengths, ang_f1, label='$\mathrm{f}_1$', color='xkcd:bright blue')
ax[0].plot(lengths, ang_f2, label='$\mathrm{f}_2$', color='xkcd:bright orange')
ax[1].plot(lengths, ang_f1 - ang_f2/5, label='$\mathrm{f}_1 - \mathrm{f}_2/5$', 
           color='xkcd:bright green')
ax[0].yaxis.set_major_formatter(FormatStrFormatter("%2.2f"))
ax[0].set_ylim([-1,1])
ax[0].legend(loc='best')
ax[1].set_xlabel('Arm cavity length [m]')
ax[0].set_ylabel('Phase gain on reflection [deg]')
ax[1].set_ylim([-0.5,0.5])
ax[1].vlines(37.79, -0.5,0.5,linestyle='--',color='xkcd:charcoal')
ax[1].set_ylabel('$\Phi_{\mathrm{f_1}} - \Phi_{\mathrm{f_2}}/5$ [deg]')
fig.subplots_adjust(wspace=0.07, hspace=0.07, top=0.92)
fig.suptitle('Phase gain on reflection (180 deg offset subtracted) for $\mathrm{f}_{1}$ and $\mathrm{f}_{2}$ sidebands');
if saveFigs:
    fig.savefig('Figures/armSBreflPhase.pdf')

#### Reflectivity of the arm cavity for $2f_{1}$ and $2f_{2}$ 
 - The $3f$ locking scheme relies on the $2f_{1}$ and $2f_{2}$ components being reflected from the arms with reflectivity near unity [See K. Arai Thesis, 4.1.1].
 - The $2f$ fields aren't resonant inside the arm cavity either.
 - So in order for the amount of $2f$ light at the symmetric port to be more or less independent of the arm cavity microscopic length, we wish for the reflectivity of the arm cavity for the $2f$ component to be fairly independent of the latter.

In [ ]:
# Some practical lengths given 40m VEA constraints
lengths = np.linspace(37.5,40,20000)
rr_2f1 = rArm(t_ITM,t_ETM, r_ITM, r_ETM, lengths, 2*f1)
rr_2f2 = rArm(t_ITM,t_ETM, r_ITM, r_ETM, lengths, 2*f2)
fig, ax = plt.subplots(2,1,sharex=True, figsize=(12,8))
ax[0].plot(lengths, np.real(rr_2f1), label='$2 \mathrm{f}_1$', color='xkcd:bright blue', alpha=0.55)
ax[1].plot(lengths, np.imag(rr_2f1), label='$2 \mathrm{f}_1$', color='xkcd:bright blue', alpha=0.55)
ax[0].plot(lengths, np.real(rr_2f2), label='$2 \mathrm{f}_2$', color='xkcd:bright orange', alpha=0.55)
ax[1].plot(lengths, np.imag(rr_2f2), label='$2 \mathrm{f}_2$', color='xkcd:bright orange', alpha=0.55)
ax[0].legend(loc='best')
ax[1].set_xlabel('Arm cavity length [m]')
ax[0].set_ylabel('Real part of $r_{\mathrm{arm}}$')
ax[1].set_ylabel('Imaginary part of $r_{\mathrm{arm}}$')
ax[0].vlines(37.795, -1,1,linestyle='--',color='xkcd:charcoal')
ax[1].vlines(37.795, -1,1,linestyle='--',color='xkcd:charcoal')
ax[0].set_ylim([-1.01,-0.95])
ax[1].set_ylim([-0.05,0.05])
fig.subplots_adjust(wspace=0.07, hspace=0.07, top=0.92)
fig.suptitle('Arm cavity reflectivity for $2 \mathrm{f}_{1}$ and $2 \mathrm{f}_{2}$ fields');
if saveFigs:
    fig.savefig('Figures/armSBreflectivity.pdf')

#### Dependence of amount of $2f$ power at REFL for various arm lengths, macroscopic detunings
The detailed deriviation is found in the section on Schnupp Asymmetry, but the reflectivity of the full IFO is given by:
$$r_{\mathrm{DRFPMI}}^{\mathrm{sym} \rightarrow \mathrm{sym}} = -r_p + \frac{t_p^2 (|r_A| e^{-i(2\phi_{\mathrm{PRC}}-\theta)}\cos(2 \phi_-) - |r_A|^2 r_S e^{-i(2\phi_{\mathrm{PRC}} + 2\phi_{\mathrm{SRC}} - 2\theta)})}{1 + |r_A|^2r_Pr_S e^{-i(2\phi_{\mathrm{PRC}} + 2\phi_{\mathrm{SRC}}-2\theta)} - |r_A|(r_P e^{-i(2\phi_{\mathrm{PRC}}-\theta)} + r_S e^{-i(2\phi_{\mathrm{SRC}}-\theta)})\cos(2\phi_-)},$$

where the symbols have the following meaning:

$$ L_{\mathrm{PRC}} = L_{\mathrm{PRM} \rightarrow \mathrm{BS}} + \frac{l_{\mathrm{BS} \rightarrow \mathrm{ITMY}} + l_{\mathrm{BS} \rightarrow \mathrm{ITMX}}}{2}$$
$$ L_{\mathrm{SRC}} = L_{\mathrm{SRM} \rightarrow \mathrm{BS}} + \frac{l_{\mathrm{BS} \rightarrow \mathrm{ITMY}} + l_{\mathrm{BS} \rightarrow \mathrm{ITMX}}}{2}$$
$$ l_{\mathrm{schnupp}} = l_{\mathrm{BS} \rightarrow \mathrm{ITMY}} - l_{\mathrm{BS} \rightarrow \mathrm{ITMX}}$$
$$ r_{\mathrm{A}} = |r_{\mathrm{A}}|e^{i \theta}$$
$$ \phi_{\mathrm{PRC}} = \frac{\omega L_{\mathrm{PRC}}}{c} $$
$$ \phi_{\mathrm{SRC}} = \frac{\omega L_{\mathrm{SRC}}}{c} $$
$$ \phi_{-} = \frac{\omega l_{\mathrm{schnupp}}}{2c} $$

In [ ]:
def rArm_det(ti, te, ri, re, L, f, phiC):
    '''
    More general form of the earlier defined function,
    such that it allows for the cavity to be detuned.
    '''
    phi = phiC + (2*np.pi*f*L/scc.c)
    return (-ri + re*(ti**2 + ri**2)*np.exp(-2*1j*phi))/(1 - ri*re*np.exp(-2*1j*phi))

def ifo_REFL(rp, rs, tp, ts, ti, te, ri, re, Larm, f, phiC, theta1, theta2):
    # Calculate arm reflectivity
    phi = phiC + (2*np.pi*f*Larm/scc.c)
    rA = (-ri + re*(ti**2 + ri**2)*np.exp(-2*1j*phi))/(1 - ri*re*np.exp(-2*1j*phi))
    mag = np.abs(rA)
    ph = np.angle(rA)
    # Set the PRC length to be resonant for both sidebands, i.e. 6.753m
    Lprc = (theta1/2/np.pi)*scc.c/2/11.066209e6
    phiPRC = 2*np.pi*f*Lprc/scc.c
    # Set the SRC length such that f2 and carrier are resonant
    Lsrc = (1 + theta2/2/np.pi)*scc.c/2/(5*11.066209e6)
    phiSRC =  2*np.pi*f*Lsrc/scc.c
    # Schnupp asy
    phiSchnupp = (2.319e-2)*2*np.pi*f/2/scc.c
    # Calculate the amplitude reflectivity
    num = tp**2 * (mag*np.exp(-1j*(2*phiPRC - ph))*np.cos(phiSchnupp) - mag**2 * rs * np.exp(-1j*(2*phiPRC + 2*phiSRC - 2*ph)))
    den = 1 + mag**2 * rp * rs * np.exp(-1j*(2*phiPRC + 2*phiSRC - 2*ph)) - mag*np.cos(phiSchnupp)*(rp*np.exp(-1j*(2*phiPRC-ph)) + rs*np.exp(-1j*(2*phiSRC-ph)))
    return -rp + num/den

lengths = np.linspace(37.5,40,500)
detunings = np.linspace(-10./532, 10./532, 500)
xx, yy = np.meshgrid(lengths, detunings)
theta1 = np.angle(rArm_det(t_ITM,t_ETM, r_ITM, r_ETM, xx, f1, np.zeros((500,500))))
theta2 = np.angle(rArm_det(t_ITM,t_ETM, r_ITM, r_ETM, xx, f2, np.zeros((500,500))))
rr_f1 = ifo_REFL(r_PRM, r_SRM, t_PRM, t_SRM, t_ITM, t_ETM, r_ITM, r_ETM, lengths, 2*f1, detunings, theta1, theta2)
rr_f2 = ifo_REFL(r_PRM, r_SRM, t_PRM, t_SRM, t_ITM, t_ETM, r_ITM, r_ETM, lengths, 2*f2, detunings, theta1, theta2)                     

In [ ]:
# This cell takes ~30 seconds to execute for a 500x500 grid
from scipy.special import jv
Pref_2f1 = (jv(2,0.18)**2) * np.abs(rr_f1)**2
Pref_2f2 = (jv(2,0.23)**2) * np.abs(rr_f2)**2

fig, ax = plt.subplots(2,1,figsize=(16,12), sharex=True)
heatmap1 = ax[0].pcolor(xx,532*yy,1e6*Pref_2f1, cmap='inferno',rasterized=True)
heatmap2 = ax[1].pcolor(xx,532*yy,1e6*Pref_2f2, cmap='inferno',rasterized=True)
c1=fig.colorbar(heatmap1, 
                pad=0.01, ax=ax[0], format='%2.1f')
c2=fig.colorbar(heatmap2, 
                pad=0.01, ax=ax[1], format='%2.1f')
c1.set_label('Reflected $2f_1$ power ($\gamma = 0.18$) [$\mu$ W]',rotation=270, labelpad=25, fontsize=20)
c2.set_label('Reflected $2f_2$ power ($\gamma = 0.22$) [$\mu$ W]',rotation=270, labelpad=25, fontsize=20)
ax[0].vlines(37.795, -10,10,linestyle='--',color='xkcd:periwinkle')
ax[1].vlines(37.795, -10,10,linestyle='--',color='xkcd:periwinkle')
ax[1].set_xlabel('Arm length [m]')
ax[0].set_ylabel('CARM detuning [nm]')
ax[1].set_ylabel('CARM detuning [nm]')
fig.suptitle('$2f$ power reflectivity to symmetric port dependence on arm length and detuning')
fig.subplots_adjust(left=0.07, right=0.95, top=0.95);
if saveFigs:
    fig.savefig('Figures/2fReflectivity.pdf')

#### Simultaneous resonance of $f_{1}$ and $f_{2}$ sidebands in the PRC
 - Having chosen the macroscopic arm length of $\mathrm{L_{arm}} = 37.795 \mathrm{m}$, we can now tune the SRC and PRC lengths to satisfy various resonance conditions.
 - Using this arm length, we can calculate the (complex) reflectivity of the arm ($r_{\mathrm{arm}} = |r_{\mathrm{arm}}|e^{i\theta}$).
 - Then we use the expression for circulating field in the FP cavity:
 $$\vec{\mathrm{E}}_{\mathrm{circ}} = \vec{\mathrm{E}}_{\mathrm{in}}\frac{t_{\mathrm{PRM}}}{1-r_{\mathrm{PRM}}r_{\mathrm{arm}}e^{-i \frac{\omega}{\nu_{\mathrm{FSR}}}}},$$ and choose the macroscopic cavity length that maximizes the circulating power for the $f_{1}$ and $f_{2}$ components for the desired operating point of the carrier.
 - Note that $\omega = \omega_0$ for the carrier, and $\omega = \omega_0 \pm \Omega_i$ for the sideband fields.
 - The correction due to the complex arm reflectivity is that we choose $\mathrm{L_{PRC}} =  (N+\frac{\theta_1}{2\pi})\frac{c}{2f_1}$, $N \in \mathbb{Z} ^+$ instead of $\mathrm{L_{PRC}} =  (N+\frac{1}{2})\frac{c}{2f_1}$, $N \in \mathbb{Z} ^+$, where $\theta_1$ is the argument of the complex arm reflectivity for the $f_1$ sideband. 
 - For the 40m PonderSqueeze experiment, we propose k = 0 and $\theta_1 \approx 180.5^\circ$ and $\theta_2 = 182.5^\circ$, such that $L_{\mathrm{PRC}} = 6.753 \mathrm{m}$.

In [ ]:
larm = 37.795
r_f1 = rArm(t_ITM,t_ETM, r_ITM, r_ETM, larm, f1)
r_f2 = rArm(t_ITM,t_ETM, r_ITM, r_ETM, larm, f2)
r_carr = rArm(t_ITM,t_ETM, r_ITM, r_ETM, larm, 0.)

def gPRC(tp, rp, rarm, L, f):
    '''
    Computes circulating power in the PRC, with compound
    mirror formed by the arm cavity, for a sideband offset at frequency f.
    '''
    FSR = scc.c / (2*L) 
    return (tp/(1 - rp*rarm*np.exp(-1j*2*np.pi*f/FSR)))
lengths = np.linspace(6.7, 6.8, 1000)
gPRC_f1 = gPRC(t_PRM, r_PRM, r_f1, lengths, f1)
gPRC_f2 = gPRC(t_PRM, r_PRM, r_f2, lengths, f2)
gPRC_carr = gPRC(t_PRM, r_PRM, r_carr, lengths, 0.)
f1Max = lengths[np.argmax(np.abs(gPRC_f1)**2)]
f2Max = lengths[np.argmax(np.abs(gPRC_f2)**2)]
print('Maximum buildup for f1 sideband is for Lprc = {0:.4f}'.format(f1Max))
print('Maximum buildup for f2 sideband is for Lprc = {0:.4f}'.format(f2Max))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,8))
ax.plot(lengths, 10*np.log10(0.01*np.abs(gPRC_f1)**2), 
        label='$\mathrm{f_1} ( \gamma = 0.2)$',color='xkcd:bright blue')
ax.plot(lengths, 10*np.log10(0.01*np.abs(gPRC_f2)**2), 
        label='$\mathrm{f_2} ( \gamma = 0.2)$',color='xkcd:bright orange')
ax.plot(lengths, 10*np.log10(np.abs(gPRC_carr)**2), 
        label='$\mathrm{f_0}$',color='xkcd:bright green')
ax.vlines(6.753,-20,20,linestyle='--',color='xkcd:charcoal')
ax.set_ylabel('Circulating power for 1W input [dB]')
ax.set_xlabel('PRC macroscopic length [m]')
ax.legend(loc='upper right')
fig.suptitle('Setting PRC macroscopic length')
fig.subplots_adjust(left=0.07, right=0.95, top=0.92)
if saveFigs:
    fig.savefig('Figures/PRCmacro.pdf')

#### Setting the SRC macroscopic length
 - For PonderSqueeze, we need the carrier to be resonant in the SRC (when the arms are resonant for the carrier).
 - Actually, there is a small detuning for the carrier, of roughly $0.01^\circ$ 
 - $f_{2}$ sideband has to be resonant.
 - $f_{1}$ sideband has to be non-resonant.

In [ ]:
def gSRC(tarm, rarm, rs, L, f):
    '''
    Computes circulating power in the SRC, with compound
    mirror formed by the arm cavity, for a sideband offset at frequency f.
    '''
    FSR = scc.c / (2*L) 
    return (tarm/(1 - rarm*rs*np.exp(-1j*(2*np.pi*f/FSR))))
lengths = np.linspace(1, 5, 10000)
gSRC_f1 = gSRC(np.sqrt(1 - r_f1**2 - L_ITM - L_ETM), r_f1, r_SRM, lengths, f1)
gSRC_f2 = gSRC(np.sqrt(1 - r_f2**2 - L_ITM - L_ETM), r_f2, r_SRM, lengths, f2)
gSRC_carr = gSRC(np.sqrt(1 - r_carr**2 - L_ITM - L_ETM), r_carr, r_SRM, lengths, 0)

fig, ax = plt.subplots(1,1,figsize=(12,8))
ax.plot(lengths, 20*np.log10(0.01*np.abs(gSRC_f1)**2), 
        label='$\mathrm{f_1} ( \gamma = 0.2)$',color='xkcd:bright blue')
ax.plot(lengths, 20*np.log10(0.01*np.abs(gSRC_f2)**2), 
        label='$\mathrm{f_2} ( \gamma = 0.2)$',color='xkcd:bright orange')
ax.plot(lengths, 20*np.log10(np.abs(gSRC_carr)**2), 
        label='$\mathrm{f_0}$',color='xkcd:bright green')
ax.set_ylabel('Circulating power for 1W input [dB]')
ax.set_xlabel('SRC macroscopic length [m]')
ax.yaxis.set_major_formatter(FormatStrFormatter('%2d'))
ax.vlines(lengths[np.argmax(20*np.log10(np.abs(gSRC_f2)**2))],-100,20,linestyles='--',color='xkcd:charcoal')
ax.legend(loc='upper right')
fig.suptitle('Setting SRC macroscopic length')
fig.subplots_adjust(left=0.07, right=0.95, top=0.92)
print('Maximum buildup for f2 sideband happens for an SRC length of {0:.4f} m'.format(lengths[np.argmax(20*np.log10(np.abs(gSRC_f2)**2))]))
if saveFigs:
    fig.savefig('Figures/SRCmacro.pdf')

#### Schnupp asymmetry
Having set the resonance conditions of the PRC and SRC, we wish for the Michelson transmission for $f_2$ to be large, but for $f_1$ to be small. Basically, we would like to critically couple the $f_2$ sideband to the dark port. Because of the SRM, we cannot simply consider the transmissivity of the Michelson. Furthermore, the arm reflectivity is in general a complex number, which we can write as $r_{\mathrm{arm}} = |r_{\mathrm{arm}}|e^{i\theta}$. The phase of this complex number can then be grouped with the PRC/SRC phase for the desired operating point. The expression for the transmission, for the more general case where we have arbitrary SRC tuning given by $\phi_{\mathrm{SRC}}$, is ($l_{\mathrm{sch}} = l_\mathrm{Y}-l_\mathrm{X}$):


$$t_{\mathrm{DRFPMI}}^{\mathrm{sym} \rightarrow \mathrm{as}} = \frac{e^{-i(\phi_{\mathrm{PRC}} + \phi_{\mathrm{SRC}} - \theta)} |r_A| t_P t_S \sin(2\phi_-)}{1 + |r_A|^2r_Pr_S e^{-i(2\phi_{\mathrm{SRC}}-\theta)} - |r_A|(r_P + r_S e^{-i(2\phi_{\mathrm{SRC}}-\theta)})\cos(2\phi_-)}.$$

*Note: Why does $\theta$ and not $2\theta$ appear in the exponent for the term in the denominator with the coefficient $|r_A|^2$ when we are assuming $r_A = |r_A|e^{i\theta}$? Because as derived earlier, the actual argument of the exponential is $-i(2\phi_{\mathrm{PRC}} + 2\phi_{\mathrm{SRC}} - 2\theta)$, and we have used the fact that $-i(2\phi_{\mathrm{PRC}} - \theta) = 0$ for the sideband being resonant in the PRC.

For the common cases:

RSE: $\phi_{\mathrm{SRC}}(\Omega) = \pi/2 + \Omega l_{\mathrm{SRC}}/c$

SR: $\phi_{\mathrm{SRC}}(\Omega) = 0 + \Omega l_{\mathrm{SRC}}/c$

In [ ]:
def tDRMI(rA,tp,ts,rp,rs,ls,wm, phiC, lSRC):
    mag = np.abs(rA)
    ph = np.angle(rA)
    phiSRC = phiC + wm*lSRC/scc.c
    return (np.exp(-1j*(phiSRC-ph))*tp*ts*np.abs(mag*np.sin(wm*ls/scc.c))/(1 + 
                                        mag**2*rp*rs*np.exp(-1j*(2*phiSRC - ph)) - 
                                        mag*(rp+rs*np.exp(-1j*(2*phiSRC - ph)))*np.cos(wm*ls/scc.c)))
lls = 1e-2 * np.linspace(0.5,40,1000)
tt_f2 = tDRMI(r_f2,np.sqrt(1e-2),np.sqrt(0.25),np.sqrt(1.-1e-2),np.sqrt(1.-0.25),lls,2*np.pi*f2,0,4.0443)
tt_f1 = tDRMI(r_f1,np.sqrt(1e-2),np.sqrt(0.25),np.sqrt(1.-1e-2),np.sqrt(1.-0.25),lls,2*np.pi*f1,0,4.0443)

fig, ax = plt.subplots(1,1,figsize=(12,8))
ax.semilogy(100*(lls), 100*np.abs(tt_f2)**2, color='xkcd:bright blue', label='$f_2$')
ax.semilogy(100*(lls), 100*np.abs(tt_f1)**2, color='xkcd:bright orange', label='$f_1$')
ax.legend(loc='best')
ax.set_ylabel('Power transmissivity [\%]')
ax.set_xlabel('Schnupp asymmetry [cm]')
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.vlines(100*lls[np.argmax(np.abs(tt_f2)**2)],0,100,linestyles='--',color='xkcd:charcoal')
fig.suptitle('Setting Schnupp asymmetry')
fig.subplots_adjust(left=0.07, right=0.95, top=0.92)
maxT_f2 = np.max(100*np.abs(tt_f2)**2) 
print('Optimal Schnupp asymmetry is {0:.3f} cm for which f2 transmission to dark port is {1:.3f} %.'.format(100*lls[np.argmax(np.abs(tt_f2)**2)], 
                                                                                                  maxT_f2))
print('Ratio of f1/f2 power buildup in the SRC is {0:.2E}'.format(np.abs(tt_f1[np.argmax(np.abs(tt_f2)**2)]/tt_f2[np.argmax(np.abs(tt_f2)**2)])**2))
if saveFigs:
    fig.savefig('Figures/schnupp_f2Trans.pdf')

In [ ]:
lSRCs = np.linspace(2.5,4.2,1000)
xx, yy = np.meshgrid(lls,lSRCs)
pSRC = tDRMI(r_f2,np.sqrt(1e-2),np.sqrt(0.1),np.sqrt(1.-1e-2),np.sqrt(1.-0.1),xx,2*np.pi*f2,0,yy)

In [ ]:
# Adjust the ranges here as you see fit - for a wider range, you can see that there are two Schnupp asy regimes that would work.
fig, ax = plt.subplots(1,1,figsize=(16,12))
heatmap = ax.contourf(100*xx,yy,1e3*0.01*np.abs(pSRC)**2,100, cmap='inferno',
                     vmin=0.9*np.min(1e3*0.01*np.abs(pSRC)**2), 
                      vmax=np.max(1e3*0.01*np.abs(pSRC)**2))
for hh in heatmap.collections:
    hh.set_rasterized(True)
levels = [2, 4, 6, 8]
C1 = ax.contour(100*xx, yy, 1e3*0.01*np.abs(pSRC)**2,levels,colors='xkcd:periwinkle', linewidths=2)
c1=fig.colorbar(heatmap, format='%3d', 
                pad=0.01)
c1.set_label('Transmitted $f_2$ power through SRM assuming $\gamma_{f_2}=0.2$ [mW]',rotation=270, labelpad=25, fontsize=20)
ax.clabel(C1, colors = 'xkcd:periwinkle', fmt = '%1d', fontsize = 18)
ax.set_xlabel('Schnupp asymmetry [cm]')
ax.set_ylabel('SRC length [m]')
fig.suptitle('$f_2$ transmission to dark port dependence on Schnupp asymmetry and SRC length')
fig.subplots_adjust(left=0.07, right=0.95, top=0.95)
if saveFigs:
    fig.savefig('Figures/schnuppHeatMap.pdf')

#### HoMs and mode-matching between the RC and arm cavities

To summarize the optimal cavity lengths:
 - $L_{\mathrm{arm}} = 37.795 $ m
 - $L_{\mathrm{PRC}} = 6.753 $ m
 - $L_{\mathrm{SRC}} = 4.044 $ m
 - $l_{\mathrm{schnupp}} = 2.319 $ cm

For the recycling cavities, criteria motivating the the choice of RoC of the mirrors are:
 - Stability of the RCs (i.e. sufficient TMS).
 - Good mode-matching between the arm mode and the RC mode.
 - Avoid HoMs resonating in the arms.
 - Curvature of all optics forming the (folded) cavities should be ~150 m, which is probably a number that manufacturers can meet without problems (since this corresponds to a sagitta of 4.83 $\mu$m for a 3-inch optic, 2.15 $\mu$m for a 2-inch optic). Of course the more important spec is what the tolerance is on the RoC (and equivalently the sag).

The calculation is identical to that done for determining the RC lengths and Schnupp Asymmetry, the main difference being the addition of the Gouy phase for the higher order HG gaussian modes, and that the length tuning is microscopic rather than macroscopic.

In order to calculate the round trip Gouy phase, we follow the derivation of T1300189. If the **round trip** ABCD matrix for a cavity (whose individual elements have an ABCD matrix given by $\mathcal{M}_i$) is given by
$$ M_{\mathrm{RT}} = \prod_{i}\mathcal{M}_{i} = \begin{bmatrix} A & B \\ C & D \end{bmatrix}, $$
the round trip Gouy Phase is given by 
$$ \zeta = 2 \cos^{-1} \left ( \mathrm{sgn} B \sqrt{\frac{A+D+2}{4}} \right ). $$

#### Target mode
The arm cavities define the IFO mode. The ITMs are flat, and the ETMs have RoC of 60.2 m. The arm cavity length is 37.795m. Per Siegman Eq 19.3, 
$$g_{\mathrm{ITM}} = 1 - \frac{L_{\mathrm{arm}}}{R_{\mathrm{ITM}}} \text{  ,  } g_{\mathrm{ETM}} = 1 - \frac{L_{\mathrm{arm}}}{R_{\mathrm{ETM}}}. $$
For this cavity, the waist is on the ITM, and per Siegman Eq 19.6, has a size
$$ w_o^2 = \frac{L_{\mathrm{arm}}\lambda}{\pi} \sqrt{\frac{g_{\mathrm{ITM}} g_{\mathrm{ETM}}(1-g_{\mathrm{ITM}}g_{\mathrm{ETM}}) }{(g_{\mathrm{ITM}} + g_{\mathrm{ETM}} -2g_{\mathrm{ITM}}g_{\mathrm{ETM}})^2}}. $$

*Note that $w$ is a spot radius.

Setting the z-coordinate of the ITM to be $z=0$, this mode can be propagated back to the PRM and SRM taking into account the intervening folding optics, which then tells us what curvature is required on the PRM and SRM. We assume that the approximate position of the folding mirrors in the PRC remain the same. So, in the PRC, the optical elements are:

| Component | z-coordinate [m] | RoC [m] |
| --- | --- | --- |
| ITM | 0 | 8000 |
| BS  | 2.33 | 5625 |
| PR3 | 2.73 | 1000 |
| PR2 | 4.82 | -200 |
| PRM | 6.753 | 70 |

For the SRC, given the macroscopic length and practical layout concerns inside the chambers, only 1 folding mirror is required, called SR2. There are actually a few options for routing the output beam, and this calculation may have to be updated depending on whether we choose to bring the AS beam EAST (to ITMY table), SOUTH (to ITMX table), or WEST (to IMC table).

| Component | z-coordinate [m] | RoC [m] |
|:---------:|:----------------:|:-------:|
| ITM | 0 | 8000 |
| BS  | 2.33 | 5625 |
| SR2 | 2.53 | -600 |
| SRM | 4.044 | 128 |

In [ ]:
def ITMwaist(Roc_ITM, Roc_ETM, Larm, lamb=1064e-9):
    g1 = 1-Larm/Roc_ITM
    g2 = 1-Larm/Roc_ETM
    return np.sqrt((Larm*lamb/np.pi)*np.sqrt((g1*g2*(1-g1*g2))/(g1 + g2 - 2*g1*g2)**2)), np.sqrt((Larm*lamb/np.pi)*np.sqrt((g1)/(g2*(1-g1*g2)))), 

waist_40m, size_ETM = ITMwaist(8000, 60.2, 37.795)
zR_arm = np.pi*waist_40m**2/1064e-9
print('The 40m arm cavity waist size is {} mm on the ITM. Spot size on the ETM is {} mm. Rayleigh range is {} m'.format(round(1e3*waist_40m,3)
                                                                                                                        , round(1e3*size_ETM,3), 
                                                                                                                       round(zR_arm,3)))

#### RoC, tolerance, and sag
In terms of manufacturing tolerances, the main point is - how precisely can the manufacturer measure the curvature of the fabricated optic? Since Fizeau interferometry is the method of choice to measure the RoC, the equivalent question is what is the precision with which the sagitta of the optic can be measured? The relationship between RoC and sag is easily derived using Pythagoras theorem, and reads:
$$ \text{sag} = R - \sqrt{R^2 - \left ( \frac{d}{2} \right )^2} $$
where $d$ is the diameter of the optic and $R$ is its RoC. In reality, $d$ may be smaller than the diameter of the optic as we don't need the clear aperture of the optic to be the entire optic - e.g. clear aperture of 80% of the full diameter is often found in LIGO optics specs. For us, the optics are either 2-inch (folding optics) or 3-inch (PRM/SRM) in diameter.

#### Gaussian beam propagation and mode overlap.
Now we are in a position to propagate the gaussian beam and determine the wavefront curvature at the location of the PRM and SRM. To do so, we use equatios (15)-(17) of T1300189. The important matrices are those for free space propagation and reflection from a curved mirror. These are given by:
$$ \mathcal{M}_{\mathrm{free-space}} = \begin{bmatrix} 1 & L \\ 0 & 1 \end{bmatrix}$$
and 
$$ \mathcal{M}_{\mathrm{mirror}} = \begin{bmatrix} 1 & 0 \\ -\frac{2}{R_{\pm}} & 1 \end{bmatrix}, R_{\pm} = R(\cos\theta)^{\pm1}.$$
Note that in this convention, for a *convex* mirror (i.e. one that causes a beam to diverge), the radius of curvature $R$ is *negative*. 

To identify a candidate PRM and SRM curvature, simply use ABCD matrices to propagate the beam with the waist at the ITM to the position of the PRM/SRM and evaluate the beam size and curvature at that location. The intervening folding mirrors are assumed to be positioned approximately where they are now (since we know that to be a practical VEA layout). For the large angle-of-incidence folding mirrors, to reduce the impact of astigmatism, I constrain the RoCs to the range 850 m - 1150 m, which is what the new folding mirrors we are getting are spec'd for.

Finally, to evaluate the mode-overlap between the recycling-cavity eigenmode and the arm-cavity eigenmode, I use Equation (6) from T1300364:
$$|\mathcal{C}|^2 = 4\sqrt{\frac{z_R^{\mathrm{arm}} z_{R_x}^{\mathrm{RC}} z_R^{\mathrm{arm}} z_{R_y}^{\mathrm{RC}}}{\left [ \left ( z^{\mathrm{arm}} - z_x^{\mathrm{RC}}  \right )^2 + \left ( z_{R}^{\mathrm{arm}} + z_{R_x}^{\mathrm{RC}}  \right )^2 \right ]  \left [ \left ( z^{\mathrm{arm}} - z_y^{\mathrm{RC}}  \right )^2 + \left ( z_{R}^{\mathrm{arm}} + z_{R_y}^{\mathrm{RC}}  \right )^2 \right ] }},  $$

with the $q-\text{parameter}$ being defined as $q_{\mathrm{TEM}_{00}} = z + iz_R$. Astigmatism is only relevant for the RCs, which have non-normal incidence on the folding mirrors. By setting the waist of the arm-cavity mode at the ITM ($z^{\mathrm{arm}} = 0$), the above equation can be simplified to 
$$|\mathcal{C}|^2 = 4\sqrt{\frac{z_R^{\mathrm{arm}} z_{R_x}^{\mathrm{RC}} z_R^{\mathrm{arm}} z_{R_y}^{\mathrm{RC}}}{\left [ \left (  z_x^{\mathrm{RC}}  \right )^2 + \left ( z_{R_x}^{\mathrm{RC}} + z_{R}^{\mathrm{arm}}  \right )^2 \right ]  \left [ \left (  z_y^{\mathrm{RC}}  \right )^2 + \left (  z_{R_y}^{\mathrm{RC}} + z_{R}^{\mathrm{arm}}  \right )^2 \right ] }}.  $$

The Rayleigh range is defined as $z_R = \frac{\pi w_0^2}{\lambda}$. The arm-cavity mode is taken as the input mode for the RC, and the *round trip* ABCD matrix for the RC is used to determine the waist location and Rayleigh range for the RC using the relations:
$$ z^{\mathrm{RC}} = \mathfrak{Re}\left [ \frac{A - D \pm \sqrt{(A+D)^2 -4}}{2C} \right ], $$ and
$$ z_R^{\mathrm{RC}} = \mathfrak{Im} \left [ \frac{A - D \pm \sqrt{(A+D)^2 -4}}{2C}] \right ]$$

In [ ]:
def M_freeSpace(L):
    return np.matrix([[1,L],[0,1]])
def M_mirror(R,theta=0,plane='vert'):
    if plane=='vert':
        Reff = R*np.cos(np.deg2rad(theta))
        M = np.matrix([[1,0],[-2/Reff,1]])
    elif plane=='hor':
        Reff = R*np.cos(np.deg2rad(theta))**-1
        M = np.matrix([[1,0],[-2/Reff,1]])
    else:
        print('Unknown plane, please use "vert" or "hor" ')
    return M
def beamDia(M, win, rin, lamb=1064e-9):
    A = M[0,0]
    B = M[0,1]
    C = M[1,0]
    D = M[1,1]
    return np.sqrt(win**2*(A + B/rin)**2 + B**2 * lamb**2/(np.pi**2 * win**2))
def wavCurv(M, win, rin, lamb=1064e-9):
    A = M[0,0]
    B = M[0,1]
    C = M[1,0]
    D = M[1,1]
    num = (C + D/rin)*(A+B/rin) + B*D*lamb**2/(np.pi**2 * win**4)
    den = (A + B/rin) + B**2*lamb**2/(np.pi**2 * win**4)
    return den/num
def rtGouy(Mrt):
    A = Mrt[0,0]
    B = Mrt[0,1]
    C = Mrt[1,0]
    D = Mrt[1,1]
    return np.sign(B)*np.arccos((A+D)/2)
def modeOverlap(Mrt_x, Mrt_y, zR_arm=29.128):
    # Assumes an arm Rayleigh range of 29.128m.
    A_x = Mrt_x[0,0]
    B_x = Mrt_x[0,1]
    C_x = Mrt_x[1,0]
    D_x = Mrt_x[1,1]
    A_y = Mrt_y[0,0]
    B_y = Mrt_y[0,1]
    C_y = Mrt_y[1,0]
    D_y = Mrt_y[1,1]
    z_x = np.real((A_x - D_x + cmath.sqrt((A_x + D_x)**2 -4))/(2*C_x))
    z_y = np.real((A_y - D_y + cmath.sqrt((A_y + D_y)**2 -4))/(2*C_y))
    zR_x = np.abs(np.imag((A_x - D_x + cmath.sqrt((A_x + D_x)**2 -4))/(2*C_x)))
    zR_y = np.abs(np.imag((A_y - D_y + cmath.sqrt((A_y + D_y)**2 -4))/(2*C_y)))
    return 4*cmath.sqrt( (zR_arm*zR_arm*zR_x*zR_y) / ((z_x**2 + (zR_x + zR_arm)**2)*(z_y**2 + (zR_y + zR_arm)**2) ))

In [ ]:
def plotBeamProfile(seedR, seedC, mirrorLocs, mirrorCurvs, mirrorAngs, dZ, zMax):
    comp = np.vstack((mirrorLocs,mirrorCurvs,mirrorAngs))
    comp = comp[:,np.argsort(comp[0,:])]
    mirrorLocs_sorted = comp[0,:]
    mirrorCurvs_sorted = comp[1,:]
    mirrorAngs_sorted = comp[2,:]
    M = np.eye(2)
    zTot = 0.
    zCum = []
    w = []
    r = []
    while zTot < zMax:
        if len(mirrorLocs_sorted)==0:
            pass
        elif zTot > mirrorLocs_sorted[0]:
            M = M_mirror(mirrorCurvs_sorted[0],mirrorAngs_sorted[0],plane='vert')*M
            print('Deleting mirror with RoC {}'.format(mirrorCurvs_sorted[0]))
            mirrorLocs_sorted = np.delete(mirrorLocs_sorted,0)
            mirrorCurvs_sorted = np.delete(mirrorCurvs_sorted,0)
            mirrorAngs_sorted = np.delete(mirrorAngs_sorted,0)            
        M = M_freeSpace(dZ) * M #Propagate the beam
        zCum.append(zTot)
        w.append(beamDia(M, seedR, seedC))
        r.append(wavCurv(M, seedR, seedC))
        zTot += dZ
    return np.array(zCum), np.array(w), np.array(r)

In [ ]:
# To minimize the effect of astigmatism, let's stick to the new PR3 spec'd to be 1000 +/- 150m
rr_PR2 = np.linspace(-300,-100,100)
rr_PR3 = np.linspace(850,1150,100)
xx, yy = np.meshgrid(rr_PR2,rr_PR3)
dias_vv = np.zeros((100,100))
curvs_vv = np.zeros((100,100))
dias_hh = np.zeros((100,100))
curvs_hh = np.zeros((100,100))
for ii, jj in enumerate(rr_PR2):
    for kk, ll in enumerate(rr_PR3):
        matPRC = M_freeSpace(1.933)*M_mirror(jj,2.1)*M_freeSpace(2.09)*M_mirror(ll,41.1)*M_freeSpace(0.4)*M_mirror(5625,45)*M_freeSpace(2.33)
        dias_vv[ii,kk] = 1e3*beamDia(matPRC, 3.141e-3, 8000)
        curvs_vv[ii,kk] = wavCurv(matPRC, 3.141e-3, 8000)
        
for ii, jj in enumerate(rr_PR2):
    for kk, ll in enumerate(rr_PR3):
        matPRC = M_freeSpace(1.933)*M_mirror(jj,2.1,plane='hor')*M_freeSpace(2.09)*M_mirror(ll,41.1,plane='hor')*M_freeSpace(0.4)*M_mirror(5625,45)*M_freeSpace(2.33)
        dias_hh[ii,kk] = 1e3*beamDia(matPRC, 3.141e-3, 8000)
        curvs_hh[ii,kk] = wavCurv(matPRC, 3.141e-3, 8000)

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(16,12), sharex=True, sharey=True)
heatmap1 = ax[0,0].pcolor(xx,yy,dias_vv, cmap='inferno', vmin=0.9*np.min(dias_vv), vmax=1.1*np.max(dias_vv));
heatmap2 = ax[1,0].pcolor(xx,yy,curvs_vv, cmap='inferno', vmin=10, vmax=100);
heatmap3 = ax[0,1].pcolor(xx,yy,dias_hh, cmap='inferno', vmin=0.9*np.min(dias_hh), vmax=1.1*np.max(dias_hh));
heatmap4 = ax[1,1].pcolor(xx,yy,curvs_hh, cmap='inferno', vmin=-10, vmax=100);


c1=fig.colorbar(heatmap1, 
                pad=0.01, ax=ax[0,1], format='%2.1f');
c2=fig.colorbar(heatmap2, 
                pad=0.01, ax=ax[1,1], format='%2.1f');
c1.set_label('Spot size on PRM [mm]',rotation=270, labelpad=25, fontsize=20);
c2.set_label('Wavefront curvature at PRM [m]',rotation=270, labelpad=25, fontsize=20);

# Some contours for a concave PRM
levels = [50, 60, 70, 80, 90];
C1 = ax[1,0].contour(xx, yy, curvs_vv, levels,colors='xkcd:charcoal', linewidths=2);
ax[0,1].clabel(C1, colors = 'xkcd:charcoal', fmt = '%3d', fontsize = 18);
C2 = ax[1,1].contour(xx, yy, curvs_hh, levels,colors='xkcd:charcoal', linewidths=2);
ax[1,1].clabel(C2, colors = 'xkcd:charcoal', fmt = '%3d', fontsize = 18);

ax[1,0].set_xlabel('RoC of PR2 [m]');
ax[1,1].set_xlabel('RoC of PR2 [m]');

ax[0,0].set_ylabel('RoC of PR3 [m]');
ax[1,0].set_ylabel('RoC of PR3 [m]');
fig.suptitle('Spot size and wavefront curvature at PRM as a function of RoC of PR2 and PR3');
fig.subplots_adjust(left=0.07, right=0.95, top=0.9, hspace=0.08, wspace=0.08);
ax[0,0].yaxis.set_major_formatter(FormatStrFormatter("%2d"));
ax[1,0].yaxis.set_major_formatter(FormatStrFormatter("%2d"));
ax[1,0].xaxis.set_major_formatter(FormatStrFormatter("%2d"));
ax[1,1].xaxis.set_major_formatter(FormatStrFormatter("%2d"));
heatmap2.cmap.set_under('xkcd:bright blue');
heatmap2.cmap.set_over('xkcd:bright blue');
heatmap4.cmap.set_under('xkcd:bright blue');
heatmap4.cmap.set_over('xkcd:bright blue');
ax[1,0].hlines(1000,-300,-100,linestyle='--',color='xkcd:periwinkle');
ax[1,0].vlines(-200,850,1150,linestyle='--',color='xkcd:periwinkle');
ax[1,1].hlines(1000,-300,-100,linestyle='--',color='xkcd:periwinkle');
ax[1,1].vlines(-200,850,1150,linestyle='--',color='xkcd:periwinkle');
ax[0,0].set_title('Vertical mode');
ax[0,1].set_title('Horizontal mode');
if saveFigs:
    fig.savefig('Figures/PRC_ROC.pdf')

In [ ]:
# Really nail down the correct curvature for PRM
matPRC = M_freeSpace(1.933)*M_mirror(-200,2.1)*M_freeSpace(2.09)*M_mirror(1000,41.1)*M_freeSpace(0.4)*M_mirror(5625,45)*M_freeSpace(2.33)
curv_PRM_vv = wavCurv(matPRC, 3.141e-3, 8000)
matPRC = M_freeSpace(1.933)*M_mirror(-200,2.1,plane='hor')*M_freeSpace(2.09)* \
M_mirror(1000,41.1, plane='hor')*M_freeSpace(0.4)*M_mirror(5625,45,plane='hor')*M_freeSpace(2.33)
curv_PRM_hh = wavCurv(matPRC, 3.141e-3, 8000)
print('Curvature at PRM location is {} m (Vertical plane), {} m (Horizontal plane), {} m (geometric mean)'
     .format(round(curv_PRM_vv,3), round(curv_PRM_hh,3), round(np.sqrt(curv_PRM_vv*curv_PRM_hh),3)))

In [ ]:
# For the SRC, try a convex SR2 with RoC in the range 100-300m.
rr_SR2 = np.linspace(-800,-100,100)
dias_vv = np.zeros(100)
curvs_vv = np.zeros(100)
dias_hh = np.zeros(100)
curvs_hh = np.zeros(100)
for ii, jj in enumerate(rr_SR2):
    matSRC = M_freeSpace(1.514)*M_mirror(jj,43.1)*M_freeSpace(0.2)*M_mirror(5000,45)*M_freeSpace(2.33)
    dias_vv[ii] = 1e3*beamDia(matSRC, 3.141e-3, 8000)
    curvs_vv[ii] = wavCurv(matSRC, 3.141e-3, 8000)
for ii, jj in enumerate(rr_SR2):
    matSRC = M_freeSpace(1.514)*M_mirror(jj,43.1,plane='hor')*M_freeSpace(0.2)*M_mirror(5000,45)*M_freeSpace(2.33)
    dias_hh[ii] = 1e3*beamDia(matSRC, 3.141e-3, 8000)
    curvs_hh[ii] = wavCurv(matSRC, 3.141e-3, 8000)

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(16,12), sharex=True)
ax[0].plot(rr_SR2, dias_vv, color='xkcd:bright blue', label='Vertical')
ax[1].plot(rr_SR2, curvs_vv, color='xkcd:bright blue', label='Vertical')
ax[0].plot(rr_SR2, dias_hh, color='xkcd:bright orange', label='Horizontal')
ax[1].plot(rr_SR2, curvs_hh, color='xkcd:bright orange', label='Horizontal')
ax[1].set_xlabel('RoC of SR2 [m]')
ax[0].set_ylabel('Beam diameter on SRM [mm]')
ax[1].set_ylabel('Curvature of wavefront on SRM [m]')
fig.suptitle('Spot size and wavefront curvature at SRM as a function of RoC of SR2')
fig.subplots_adjust(left=0.07, right=0.95, top=0.95, hspace=0.08, wspace=0.08)
ax[0].yaxis.set_major_formatter(FormatStrFormatter("%2.2f"))
ax[1].yaxis.set_major_formatter(FormatStrFormatter("%2d"))
ax[1].xaxis.set_major_formatter(FormatStrFormatter("%2d"))
ax[0].grid(True, which='both',linestyle='--',alpha=0.4)
ax[0].legend(loc='best');
ax[1].vlines(-500,40,160,linestyle='--',color='xkcd:charcoal');
if saveFigs:
    fig.savefig('Figures/SRC_ROC.pdf')

In [ ]:
# Really nail down the correct curvature for SRM
matSRC = M_freeSpace(1.514)*M_mirror(-500,43.1)*M_freeSpace(0.2)*M_mirror(5000,45)*M_freeSpace(2.33)
curv_SRM_vv = wavCurv(matSRC, 3.141e-3, 8000)
matSRC = M_freeSpace(1.514)*M_mirror(-500,43.1,plane='hor')*M_freeSpace(0.2)*M_mirror(5000,45,plane='hor')*M_freeSpace(2.33)
curv_SRM_hh = wavCurv(matSRC, 3.141e-3, 8000)
print('Curvature at SRM location is {} m (Vertical plane), {} m (Horizontal plane), {} m (geometric mean)'
     .format(round(curv_SRM_vv,3), round(curv_SRM_hh,3), round(np.sqrt(curv_SRM_vv*curv_SRM_hh),3)))

#### Transverse mode spacing for the PRC
From the above studies, it looks like some candidate curvatures are:
 - PR2 should be convex with RoC = -200 m.
 - PR3 should be concave with RoC = 1000 m.
 - PRM should be concave with RoC = 70 m.
Try some MCMC to see what is the sensitivity of the mode-matching and TMS for these choices.

In [ ]:
# This cell takes ~2.5 minutes to run for 1e5 MC samples.
def lnprob(x, mu, icov):
    diff = x-mu
    return -np.dot(diff,np.dot(icov,diff))/2.0

# Initialize the emcee sampler
nWalkers = 20
nDim = 9 # RoC of PRM, PR2, PR3, lp1, lp2, lp3, lPR3toITM, AoIs on PR2/PR3
nPts = int(1e5)

means = np.zeros(nDim)   #The means will be 0.
cov = np.diag([0.03, 0.05, 0.15, 0.0015, 0.0014, 0.0075, 0.0013, 0.1, 0.02]) # Errors chosen given nominal values, and what is "reasonable" tolerance
cov = np.dot(cov,cov)
icov = np.linalg.inv(cov)
p0 = np.random.rand(nDim * nWalkers).reshape((nWalkers, nDim))
sampler = emcee.EnsembleSampler(nWalkers, nDim, lnprob, args=[means, icov])
pos, prob, state = sampler.run_mcmc(p0, 5000)
sampler.reset()
# Generate the perturbations
pos_final, prob_final, state_final = sampler.run_mcmc(pos, nPts)

# Define the nominal values which we will perturb
PRM_R_nom = 75.
PR2_R_nom = -200.
PR3_R_nom = 1000.
lp1_nom = 1.933
lp2_nom = 2.09
lp3_nom = 0.4
lp4_nom = 2.33
aoi_PR2_nom = 2.1
aoi_PR3_nom = 41.1

# Use the generated perturbations to evaluate the TMS
nSamples = nPts
gouy_vv = np.ones(nSamples)
gouy_hh = np.ones(nSamples)
mm = np.ones(nSamples, dtype='complex')

for jj in range(nSamples):
    #Make it a fractional change
    perturb = 1+sampler.flatchain[jj,:]
    PRM_R = perturb[0] * PRM_R_nom
    PR2_R = perturb[1] * PR2_R_nom
    PR3_R = perturb[2] * PR3_R_nom
    lp1 = perturb[3] * lp1_nom
    lp2 = perturb[4] * lp2_nom
    lp3 = perturb[5] * lp3_nom
    lp4 = perturb[6] * lp4_nom
    aoi_PR2 = perturb[7] * aoi_PR2_nom
    aoi_PR3 = perturb[8] * aoi_PR3_nom
    M_RT_vv =  M_freeSpace(lp4) * M_mirror(5625., 45.) * M_freeSpace(lp3) * M_mirror(PR3_R, aoi_PR3) * \
            M_freeSpace(lp2) * M_mirror(PR2_R, aoi_PR2) * M_freeSpace(lp1) * M_mirror(PRM_R,0.) * \
        M_freeSpace(lp1)*M_mirror(PR2_R,aoi_PR2)*M_freeSpace(lp2)*M_mirror(PR3_R,aoi_PR3)*M_freeSpace(lp3)*M_mirror(5625,45)*M_freeSpace(lp4)
    M_RT_hh =  M_freeSpace(lp4) * M_mirror(5625., 45.,plane='hor') * M_freeSpace(lp3) * M_mirror(PR3_R, aoi_PR3,plane='hor') * \
            M_freeSpace(lp2) * M_mirror(PR2_R, aoi_PR2,plane='hor') * M_freeSpace(lp1) * M_mirror(PRM_R,0.,plane='hor') * \
        M_freeSpace(lp1)*M_mirror(PR2_R,aoi_PR2,plane='hor')*M_freeSpace(lp2)*M_mirror(PR3_R,aoi_PR3,plane='hor')* \
    M_freeSpace(lp3)*M_mirror(5625,45,plane='hor')*M_freeSpace(lp4)
    
    # Calculate the parameters of interest: Gouy phase, beam size, mode-matching between PRC and arm cavity
    gouy_vv[jj] = rtGouy(M_RT_vv)
    gouy_hh[jj] = rtGouy(M_RT_hh)
    mm[jj] = modeOverlap(M_RT_vv, M_RT_hh)

In [ ]:
FSR_PRC = scc.c/2/6.753/1e6 #PRC FSR in MHz
TMS_PRC_vv = FSR_PRC*gouy_vv/2/np.pi
TMS_PRC_hh = FSR_PRC*gouy_hh/2/np.pi
modeMatch = np.real(mm)
outVars = np.vstack((TMS_PRC_vv,TMS_PRC_hh, modeMatch)).T
fig, ax = plt.subplots(np.shape(outVars)[1], np.shape(outVars)[1], figsize=(12,12));
corner.corner(outVars,
              labels=['$\\nu_{\\mathrm{TMS}}^{\\mathrm{vert}}$ [MHz]','$\\nu_{\\mathrm{TMS}}^{\\mathrm{hor}}$ [MHz]', '$|\\mathcal{C}|^2$'],
              color='xkcd:electric pink',
              show_titles=True,
              title_fmt='.4f',
              use_math_text=True,
              range=[(1.2,1.8),(1.2,1.65),(0.98,1.005)],
              bins=50,
              smooth=0.5,
              hist_kwargs  = {'linewidth':2.5},
              label_kwargs = {'fontsize':'large', 'fontweight':'bold'},
              title_kwargs = {'fontsize':'medium', 'fontweight':'bold'},
              fig=fig, rasterized=True);
for ii in ax:
    for jj in ii:
        jj.grid(True,which='both',linestyle='--',alpha=0.6)
if saveFigs:
    fig.savefig('Figures/PRC_MCMC.pdf')

In [ ]:
# Also plot the histograms for the perturbations applied
perturb = 1+sampler.flatchain
PRM_R = perturb[:,0] * PRM_R_nom
PR2_R = perturb[:,1] * PR2_R_nom
PR3_R = perturb[:,2] * PR3_R_nom
lp1 = perturb[:,3] * lp1_nom
lp2 = perturb[:,4] * lp2_nom
lp3 = perturb[:,5] * lp3_nom
lp4 = perturb[:,6] * lp4_nom
aoi_PR2 = perturb[:,7] * aoi_PR2_nom
aoi_PR3 = perturb[:,8] * aoi_PR3_nom
perturbations = np.vstack((PRM_R,PR2_R,PR3_R,
                          lp1,lp2,lp3,lp4,
                          aoi_PR2,aoi_PR3)).T
# This one doesn't need to be a CORNER plot, as the perturbations are assumed i.i.d Gaussian
titles = ['$R_{\mathrm{PRM}}$ [m]','$R_{\mathrm{PR2}}$ [m]','$R_{\mathrm{PR3}}$ [m]',\
          '$l_{\mathrm{PRM} \\rightarrow \mathrm{PR2}}$ [m]',\
          '$l_{\mathrm{PR2} \\rightarrow \mathrm{PR3}}$ [m]',\
          '$l_{\mathrm{PR3} \\rightarrow \mathrm{BS}}$ [m]',\
          '$l_{\mathrm{BS} \\rightarrow \mathrm{ITM}}$ [m]',\
          '$\\theta_{\mathrm{PR2}} [^{\circ}]$','$\\theta_{\mathrm{PR3}} [^{\circ}]$']
quantiles = np.percentile(perturbations,[0.13,2.275,15.87, 84.13, 97.725, 99.87],axis=0)
means = np.mean(perturbations,axis=0)
fig, ax = plt.subplots(3,3,figsize=(24,24));
counter = 0
for ii in ax:
    for jj in ii:
        jj.hist(perturbations[:,counter], color='xkcd:electric pink', alpha=0.7, linewidth=3,
               bins=50,rasterized=True, histtype='step',density=False);
        jj.vlines(means[counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', colors='xkcd:charcoal',label='$\mu$',alpha=0.7);
        for kk in range(len(quantiles)):
            if kk==0:
                jj.vlines(quantiles[kk, counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors='xkcd:bright green',label='$\pm 3 \sigma$',alpha=0.7);
            elif kk==1:
                jj.vlines(quantiles[kk, counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors='xkcd:bright orange',label='$\pm 2 \sigma$',alpha=0.7);
            elif kk==2:
                jj.vlines(quantiles[kk, counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors='xkcd:bright blue',label='$\pm 1 \sigma$',alpha=0.7);
            else:
                jj.vlines([quantiles[kk, counter],quantiles[kk+1, counter],quantiles[kk+2, counter],],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors=['xkcd:bright blue','xkcd:bright orange','xkcd:bright green'],alpha=0.7);    
                break  
        jj.grid(True,which='both',linestyle='--',alpha=0.6)
        jj.yaxis.set_major_formatter(FormatStrFormatter("%2.1E"))
        jj.set_xlabel(titles[counter], fontsize='large', fontweight='extra bold')
        counter+=1
# Some custom formatting
ax[0,1].xaxis.set_major_formatter(FormatStrFormatter("%3d"))
ax[0,2].xaxis.set_major_formatter(FormatStrFormatter("%3d"))
ax[0,0].legend(loc='best')
fig.suptitle('Histograms of perturbations used to generate corner plot for PRC', 
             fontsize='large',
            fontweight='extra bold');
fig.subplots_adjust(top=0.95)
if saveFigs:
    fig.savefig('Figures/PRC_dists.pdf')

#### Transverse mode spacing for the SRC
Similarly, some candidate curvatures for the SRC are:
 - SR2 should be convex with RoC = -600 m.
 - SRM should be concave with RoC = 128 m.
Try some MCMC to see what is the sensitivity of the mode-matching and TMS for these choices.

In [ ]:
# This cell takes ~2.5 minutes for 1e5 MC samples.
# Initialize the emcee sampler
nWalkers = 20
nDim = 6 # RoC of SRM, SR2, ls1, ls2, lSR2toITM, AoI on SR2
nPts = int(1e5)

means = np.zeros(nDim)   #The means will be 0.
cov = np.diag([0.08, 0.08, 0.002, 0.015, 0.0013, 0.02]) # Errors chosen given nominal values, and what is "reasonable" tolerance
cov = np.dot(cov,cov)
icov = np.linalg.inv(cov)
p0 = np.random.rand(nDim * nWalkers).reshape((nWalkers, nDim))
sampler = emcee.EnsembleSampler(nWalkers, nDim, lnprob, args=[means, icov])
pos, prob, state = sampler.run_mcmc(p0, 5000)
sampler.reset()
# Generate the perturbations
pos_final, prob_final, state_final = sampler.run_mcmc(pos, nPts)

# Define the nominal values which we will perturb
SRM_R_nom = 118.
SR2_R_nom = -500.
ls1_nom = 1.514
ls2_nom = 0.2
ls3_nom = 2.33
aoi_SR2_nom = 43.1

# Use the generated perturbations to evaluate the TMS
nSamples = nPts
SRC_gouy_vv = np.ones(nSamples)
SRC_gouy_hh = np.ones(nSamples)
SRC_mm = np.ones(nSamples, dtype='complex')

for jj in range(nSamples):
    #Make it a fractional change
    perturb = 1+sampler.flatchain[jj,:]
    SRM_R = perturb[0] * SRM_R_nom
    SR2_R = perturb[1] * SR2_R_nom
    ls1 = perturb[2] * ls1_nom
    ls2 = perturb[3] * ls2_nom
    ls3 = perturb[4] * ls3_nom
    aoi_SR2 = perturb[5] * aoi_SR2_nom
    M_RT_vv =  M_freeSpace(ls3) * M_mirror(5625., 45.) * \
            M_freeSpace(ls2) * M_mirror(SR2_R, aoi_SR2) * M_freeSpace(ls1) * M_mirror(SRM_R,0.) * \
        M_freeSpace(ls1)*M_mirror(SR2_R,aoi_SR2)*M_freeSpace(ls2)*M_mirror(5625,45)*M_freeSpace(ls3)
    
    M_RT_hh =  M_freeSpace(ls3) * M_mirror(5625., 45.,plane='hor') * \
            M_freeSpace(ls2) * M_mirror(SR2_R, aoi_SR2,plane='hor') * M_freeSpace(ls1) * M_mirror(SRM_R,0.) * \
        M_freeSpace(ls1)*M_mirror(SR2_R,aoi_SR2,plane='hor')*M_freeSpace(ls2)*M_mirror(5625,45,plane='hor')*M_freeSpace(ls3)
    
    # Calculate the parameters of interest: Gouy phase, beam size, mode-matching between PRC and arm cavity
    SRC_gouy_vv[jj] = rtGouy(M_RT_vv)
    SRC_gouy_hh[jj] = rtGouy(M_RT_hh)
    SRC_mm[jj] = modeOverlap(M_RT_vv, M_RT_hh)

In [ ]:
FSR_SRC = scc.c/2/4.044/1e6 #PRC FSR in MHz
TMS_SRC_vv = FSR_SRC*SRC_gouy_vv/2/np.pi
TMS_SRC_hh = FSR_SRC*SRC_gouy_hh/2/np.pi
SRC_modeMatch = np.real(SRC_mm)
outVars = np.vstack((TMS_SRC_vv,TMS_SRC_hh, SRC_modeMatch)).T
fig, ax = plt.subplots(np.shape(outVars)[1], np.shape(outVars)[1], figsize=(12,12));
corner.corner(outVars,
              labels=['$\\nu_{\\mathrm{TMS}}^{\\mathrm{vert}}$ [MHz]','$\\nu_{\\mathrm{TMS}}^{\\mathrm{hor}}$ [MHz]', '$|\\mathcal{C}|^2$'],
              color='xkcd:electric pink',
              show_titles=True,
              title_fmt='.4f',
              use_math_text=True,
              range=[(1.1,1.8),(1.6,2.1),(0.98,1.005)],
              bins=50,
              smooth=0.5,
              hist_kwargs  = {'linewidth':2.5},
              label_kwargs = {'fontsize':'large', 'fontweight':'bold'},
              title_kwargs = {'fontsize':'medium', 'fontweight':'bold'},
              fig=fig);
for ii in ax:
    for jj in ii:
        jj.grid(True,which='both',linestyle='--',alpha=0.6)
if saveFigs:
    fig.savefig('Figures/SRC_MCMC.pdf')

In [ ]:
# Also plot the histograms for the perturbations applied
perturb = 1+sampler.flatchain
SRM_R = perturb[:,0] * SRM_R_nom
SR2_R = perturb[:,1] * SR2_R_nom
ls1 = perturb[:,2] * ls1_nom
ls2 = perturb[:,3] * ls2_nom
ls3 = perturb[:,4] * ls3_nom
aoi_SR2 = perturb[:,5] * aoi_SR2_nom
perturbations = np.vstack((SRM_R,SR2_R,
                          ls1,ls2,ls3,
                          aoi_SR2)).T
# This one doesn't need to be a CORNER plot, as the perturbations are assumed i.i.d Gaussian
titles = ['$R_{\mathrm{SRM}}$ [m]','$R_{\mathrm{SR2}}$ [m]',\
          '$l_{\mathrm{SRM} \\rightarrow \mathrm{SR2}}$ [m]',\
          '$l_{\mathrm{SR2} \\rightarrow \mathrm{BS}}$ [m]',\
          '$l_{\mathrm{BS} \\rightarrow \mathrm{ITM}}$ [m]',\
          '$\\theta_{\mathrm{SR2}} [^{\circ}]$']
quantiles = np.percentile(perturbations,[0.13,2.275,15.87, 84.13, 97.725, 99.87],axis=0)
means = np.mean(perturbations,axis=0)
fig, ax = plt.subplots(3,2,figsize=(24,24));
counter = 0
for ii in ax:
    for jj in ii:
        jj.hist(perturbations[:,counter], color='xkcd:electric pink', alpha=0.7, linewidth=3,
               bins=50,rasterized=True, histtype='step',density=False);
        jj.vlines(means[counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', colors='xkcd:charcoal',label='$\mu$',alpha=0.7);
        for kk in range(len(quantiles)):
            if kk==0:
                jj.vlines(quantiles[kk, counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors='xkcd:bright green',label='$\pm 3 \sigma$',alpha=0.7);
            elif kk==1:
                jj.vlines(quantiles[kk, counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors='xkcd:bright orange',label='$\pm 2 \sigma$',alpha=0.7);
            elif kk==2:
                jj.vlines(quantiles[kk, counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors='xkcd:bright blue',label='$\pm 1 \sigma$',alpha=0.7);
            else:
                jj.vlines([quantiles[kk, counter],quantiles[kk+1, counter],quantiles[kk+2, counter],],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors=['xkcd:bright blue','xkcd:bright orange','xkcd:bright green'],alpha=0.7);    
                break        
        jj.grid(True,which='both',linestyle='--',alpha=0.6);
        jj.yaxis.set_major_formatter(FormatStrFormatter("%.3f"))
        jj.set_xlabel(titles[counter], fontsize='large', fontweight='extra bold')
        counter+=1
# Some custom formatting
ax[0,0].xaxis.set_major_formatter(FormatStrFormatter("%3d"))
ax[0,1].xaxis.set_major_formatter(FormatStrFormatter("%3d"))
ax[0,0].legend(loc='best');

fig.suptitle('Histograms of perturbations used to generate corner plot for SRC', 
             fontsize='large',
            fontweight='extra bold');
fig.subplots_adjust(top=0.95)
if saveFigs:
    fig.savefig('Figures/SRC_dists.pdf')

#### Short-term SRC length adjustment without getting new SRM
Here, I'm thinking about whether we can get away with using the convex G&H mirror (RoC ~-700m) as an SR2 (located approximately at the current position of SR3), and the existing SRM (RoC ~ 142 m), to realize the macroscopic SRC length of 4.04m. Below, the TMS and MM to arm cavity are evaluated for this choice of parameters.

In [ ]:
# This cell takes ~2.5 minutes for 1e5 MC samples.
# Initialize the emcee sampler
nWalkers = 20
nDim = 6 # RoC of SRM, SR2, ls1, ls2, lSR2toITM, AoI on SR2
nPts = int(1e5)

means = np.zeros(nDim)   #The means will be 0.
cov = np.diag([0.08, 0.08, 0.002, 0.015, 0.0013, 0.02]) # Errors chosen given nominal values, and what is "reasonable" tolerance
cov = np.dot(cov,cov)
icov = np.linalg.inv(cov)
p0 = np.random.rand(nDim * nWalkers).reshape((nWalkers, nDim))
sampler = emcee.EnsembleSampler(nWalkers, nDim, lnprob, args=[means, icov])
pos, prob, state = sampler.run_mcmc(p0, 5000)
sampler.reset()
# Generate the perturbations
pos_final, prob_final, state_final = sampler.run_mcmc(pos, nPts)

# Define the nominal values which we will perturb
SRM_R_nom = 142.
SR2_R_nom = -700.
ls1_nom = 1.514
ls2_nom = 0.2
ls3_nom = 2.33
aoi_SR2_nom = 43.1

# Use the generated perturbations to evaluate the TMS
nSamples = nPts
SRC_gouy_vv = np.ones(nSamples)
SRC_gouy_hh = np.ones(nSamples)
SRC_mm = np.ones(nSamples, dtype='complex')

for jj in range(nSamples):
    #Make it a fractional change
    perturb = 1+sampler.flatchain[jj,:]
    SRM_R = perturb[0] * SRM_R_nom
    SR2_R = perturb[1] * SR2_R_nom
    ls1 = perturb[2] * ls1_nom
    ls2 = perturb[3] * ls2_nom
    ls3 = perturb[4] * ls3_nom
    aoi_SR2 = perturb[5] * aoi_SR2_nom
    M_RT_vv =  M_freeSpace(ls3) * M_mirror(5625., 45.) * \
            M_freeSpace(ls2) * M_mirror(SR2_R, aoi_SR2) * M_freeSpace(ls1) * M_mirror(SRM_R,0.) * \
        M_freeSpace(ls1)*M_mirror(SR2_R,aoi_SR2)*M_freeSpace(ls2)*M_mirror(5625,45)*M_freeSpace(ls3)
    
    M_RT_hh =  M_freeSpace(ls3) * M_mirror(5625., 45.,plane='hor') * \
            M_freeSpace(ls2) * M_mirror(SR2_R, aoi_SR2,plane='hor') * M_freeSpace(ls1) * M_mirror(SRM_R,0.) * \
        M_freeSpace(ls1)*M_mirror(SR2_R,aoi_SR2,plane='hor')*M_freeSpace(ls2)*M_mirror(5625,45,plane='hor')*M_freeSpace(ls3)
    
    # Calculate the parameters of interest: Gouy phase, beam size, mode-matching between PRC and arm cavity
    SRC_gouy_vv[jj] = rtGouy(M_RT_vv)
    SRC_gouy_hh[jj] = rtGouy(M_RT_hh)
    SRC_mm[jj] = modeOverlap(M_RT_vv, M_RT_hh)

In [ ]:
FSR_SRC = scc.c/2/4.044/1e6 #PRC FSR in MHz
TMS_SRC_vv = FSR_SRC*SRC_gouy_vv/2/np.pi
TMS_SRC_hh = FSR_SRC*SRC_gouy_hh/2/np.pi
SRC_modeMatch = np.real(SRC_mm)
outVars = np.vstack((TMS_SRC_vv,TMS_SRC_hh, SRC_modeMatch)).T
fig, ax = plt.subplots(np.shape(outVars)[1], np.shape(outVars)[1], figsize=(12,12));
corner.corner(outVars,
              labels=['$\\nu_{\\mathrm{TMS}}^{\\mathrm{vert}}$ [MHz]','$\\nu_{\\mathrm{TMS}}^{\\mathrm{hor}}$ [MHz]', '$|\\mathcal{C}|^2$'],
              color='xkcd:electric pink',
              show_titles=True,
              title_fmt='.4f',
              use_math_text=True,
              range=[(1.1,1.8),(1.4,2.1),(0.98,1.005)],
              bins=50,
              smooth=0.5,
              hist_kwargs  = {'linewidth':2.5},
              label_kwargs = {'fontsize':'large', 'fontweight':'bold'},
              title_kwargs = {'fontsize':'medium', 'fontweight':'bold'},
              fig=fig);
for ii in ax:
    for jj in ii:
        jj.grid(True,which='both',linestyle='--',alpha=0.6)
if saveFigs:
    fig.savefig('Figures/SRC_MCMC_shortTerm.pdf')

In [ ]:
# Also plot the histograms for the perturbations applied
perturb = 1+sampler.flatchain
SRM_R = perturb[:,0] * SRM_R_nom
SR2_R = perturb[:,1] * SR2_R_nom
ls1 = perturb[:,2] * ls1_nom
ls2 = perturb[:,3] * ls2_nom
ls3 = perturb[:,4] * ls3_nom
aoi_SR2 = perturb[:,5] * aoi_SR2_nom
perturbations = np.vstack((SRM_R,SR2_R,
                          ls1,ls2,ls3,
                          aoi_SR2)).T
# This one doesn't need to be a CORNER plot, as the perturbations are assumed i.i.d Gaussian
titles = ['$R_{\mathrm{SRM}}$ [m]','$R_{\mathrm{SR2}}$ [m]',\
          '$l_{\mathrm{SRM} \\rightarrow \mathrm{SR2}}$ [m]',\
          '$l_{\mathrm{SR2} \\rightarrow \mathrm{BS}}$ [m]',\
          '$l_{\mathrm{BS} \\rightarrow \mathrm{ITM}}$ [m]',\
          '$\\theta_{\mathrm{SR2}} [^{\circ}]$']
quantiles = np.percentile(perturbations,[0.13,2.275,15.87, 84.13, 97.725, 99.87],axis=0)
means = np.mean(perturbations,axis=0)
fig, ax = plt.subplots(3,2,figsize=(24,24));
counter = 0
for ii in ax:
    for jj in ii:
        jj.hist(perturbations[:,counter], color='xkcd:electric pink', alpha=0.7, linewidth=3,
               bins=50,rasterized=True, histtype='step',density=False);
        jj.vlines(means[counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', colors='xkcd:charcoal',label='$\mu$',alpha=0.7);
        for kk in range(len(quantiles)):
            if kk==0:
                jj.vlines(quantiles[kk, counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors='xkcd:bright green',label='$\pm 3 \sigma$',alpha=0.7);
            elif kk==1:
                jj.vlines(quantiles[kk, counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors='xkcd:bright orange',label='$\pm 2 \sigma$',alpha=0.7);
            elif kk==2:
                jj.vlines(quantiles[kk, counter],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors='xkcd:bright blue',label='$\pm 1 \sigma$',alpha=0.7);
            else:
                jj.vlines([quantiles[kk, counter],quantiles[kk+1, counter],quantiles[kk+2, counter],],0,1,transform=jj.get_xaxis_transform(), linestyle='--', 
                          colors=['xkcd:bright blue','xkcd:bright orange','xkcd:bright green'],alpha=0.7);    
                break        
        jj.grid(True,which='both',linestyle='--',alpha=0.6);
        jj.yaxis.set_major_formatter(FormatStrFormatter("%.3f"))
        jj.set_xlabel(titles[counter], fontsize='large', fontweight='extra bold')
        counter+=1
# Some custom formatting
ax[0,0].xaxis.set_major_formatter(FormatStrFormatter("%3d"))
ax[0,1].xaxis.set_major_formatter(FormatStrFormatter("%3d"))
ax[0,0].legend(loc='best');

fig.suptitle('Histograms of perturbations used to generate corner plot for SRC', 
             fontsize='large',
            fontweight='extra bold');
fig.subplots_adjust(top=0.95)
if saveFigs:
    fig.savefig('Figures/SRC_dists_shortTerm.pdf')

#### Sensing response
Next, we'd like to get some idea of what kind of sensing response we can expect for a given RC configuration. Assume we have Photodiodes at REFL11, REFL55 and AS55. Using the Adjacency Matrix, we can extract the coupling from any port to any other port. For small modulation depths, the sideband power is well approximated by $P_{\mathrm{carrier}} \approx J_0(\gamma)P_0$, $P_{\mathrm{SB}} \approx J_{1}(\gamma)P_0$. The RF photocurrent at any given port is given by:

$$I_{\mathrm{RF}} \propto \left (\sum_{i=0}^{2} c_i(\gamma) E_i \right )^2 $$

where the subscript indexes the carrier, $f_1$ and $f_2$ sidebands and $c_i(\gamma)$ encodes the dependence on modulation depth $\gamma$. For $3f$ demodulation, we need to explicitly include the higher order terms in the Jacobi-Anger expansion instead of using the small modulation-depth approximation.

The actual error signal is demodulated down to DC (i.e. we wish to isolate the component of $I_{\mathrm{RF}}$ that oscillates at $\omega_0 \pm \Omega$). Following the approach of Mizuno and Yamaguchi (1999), in terms of the m-th element of the Adjacency Matrix $\mathcal{A}_m (\Omega, \phi)$, the sensing response is given by:

$$\Re [H_m e^{-i\theta}],$$
$$\Re \left [ (\mathcal{A}^*_m (\omega_0 -\Omega, \phi) \mathcal{A}_m (\omega_0, \phi) + \mathcal{A}^*_m (\omega_0, \phi) \mathcal{A}_m (\omega_0 + \Omega, \phi))e^{-i \theta} \right ],$$

where $\phi$ denotes the tuning of a particular degree of freedom (while the others are assumed to be at their nominal operating points) and $\theta$ denotes the demodulation phase (so basically we need to demodulate at $\theta = 0, \pi/2$ for I and Q respectively).